## Part 1

In [47]:
with open('inputTest.txt') as f:
    lines = f.readlines()
lines = [line.replace('\n', '') for line in lines]


def parse_lines(lines):
    sensor_beacon_list = []
    for line in lines:
        line = line.replace('Sensor at x=', '').replace(', y=', ',').replace(': closest beacon is at x=', ',').replace(', y=', ',')
        numbs = [int(x) for x in line.split(',')]
        sensor_beacon_list.append([[numbs[0], numbs[1]], [numbs[2], numbs[3]]])
    return sensor_beacon_list

sensor_beacon_list = parse_lines(lines)
# print(sensor_beacon_list)

def get_mins_maxs(sensor_beacon_list):
    x_min, x_max, y_min, y_max = None, None, None, None
    for sensor_beacon in sensor_beacon_list:
        sensor = sensor_beacon[0]
        beacon = sensor_beacon[1]
        if x_min is None:
            x_min = sensor[0]
            x_max = sensor[0]
            y_min = sensor[1]
            y_max = sensor[1]
        if x_min > sensor[0]:
            x_min = sensor[0]
        if x_max < sensor[0]:
            x_max = sensor[0]
        if y_min > sensor[1]:
            y_min = sensor[1]
        if y_max < sensor[1]:
            y_max = sensor[1]
        if x_min > beacon[0]:
            x_min = beacon[0]
        if x_max < beacon[0]:
            x_max = beacon[0]
        if y_min > beacon[1]:
            y_min = beacon[1]
        if y_max < beacon[1]:
            y_max = beacon[1]
    return x_min, x_max, y_min, y_max

x_min, x_max, y_min, y_max = get_mins_maxs(sensor_beacon_list)
print(x_min, x_max, y_min, y_max)
print(x_max - x_min + 1, y_max - y_min + 1)

def update_sensor_beacon_list(sensor_beacon_list, x_min, x_max, y_min, y_max):
    for sensor_beacon in sensor_beacon_list:
        sensor = sensor_beacon[0]
        beacon = sensor_beacon[1]
        sensor[0] -= x_min
        sensor[1] -= y_min
        beacon[0] -= x_min
        beacon[1] -= y_min
    return sensor_beacon_list

updated_sensor_beacon_list = update_sensor_beacon_list(sensor_beacon_list, x_min, x_max, y_min, y_max)
# print(updated_sensor_beacon_list)

def manhattan_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

def is_beacon_impossible(coor, sensor_beacon_list):
    for sensor_beacon in sensor_beacon_list:
        sensor = sensor_beacon[0]
        beacon = sensor_beacon[1]
        sensor_beacon_dist = manhattan_distance(sensor[0], sensor[1], beacon[0], beacon[1])
        sensor_coor_dist = manhattan_distance(sensor[0], sensor[1], coor[0], coor[1])
        if sensor_coor_dist <= sensor_beacon_dist:
            return True
    return False

sensors = [sensor_beacon[0] for sensor_beacon in updated_sensor_beacon_list]
beacons = [sensor_beacon[1] for sensor_beacon in updated_sensor_beacon_list]

def create_grid(x_min, x_max, y_min, y_max, sensors, beacons, sensor_beacon_list):

    grid = []
    for y in range(0, y_max-y_min + 1):
        # print the percentage of the grid that has been created
        # if y % 10 == 0:
        #     print(y/(y_max-y_min + 1))
        row = []
        for x in range(0, x_max-x_min + 1):
            if [x,y] in sensors:
                row.append('S')
            elif [x,y] in beacons:
                row.append('B')
            elif is_beacon_impossible([x,y], sensor_beacon_list):
                row.append('#')
            else:
                row.append('.')
        grid.append(row)
    return grid

def create_row(y, x_min, x_max, sensors, beacons, sensor_beacon_list):
    
    row = []
    for x in range(0, x_max-x_min + 1):
        if [x,y] in sensors:
            row.append('S')
        elif [x,y] in beacons:
            row.append('B')
        elif is_beacon_impossible([x,y], sensor_beacon_list):
            row.append('#')
        else:
            row.append('.')
    return row

def brute_force(y, sensor_beacon_list):
    count = 0
    for x in range(-50, 50):
        # log every 1000000
        if x % 1000000 == 0:
            print(x)
        if is_beacon_impossible([x,y], sensor_beacon_list):
            count += 1
    return count

brute_forced = brute_force(10, sensor_beacon_list)
print(brute_forced)





# # grid = create_grid(x_min, x_max, y_min, y_max, sensors, beacons, updated_sensor_beacon_list)
# # for row in grid:
# #     print(row)
row = create_row(10, x_min, x_max, sensors, beacons, updated_sensor_beacon_list)
print(row)

def count_impossible_beacons(y_slice):
    count = 0
    for coor in y_slice:
        if coor == '#':
            count += 1
    return count

    

impossible_beacons = count_impossible_beacons(row)
print(impossible_beacons)




    



-2 25 0 22
28 23
0
27


In [48]:
import sys
import math
from copy import deepcopy
from collections import defaultdict, deque
infile = sys.argv[1] if len(sys.argv)>1 else '15.in'
data = open("input.txt").read().strip()
lines = [x for x in data.split('\n')]

S = set()
B = set()
sum_d = 0
for line in lines:
    words = line.split()
    sx,sy = words[2],words[3]
    bx,by = words[8],words[9]
    sx = int(sx[2:-1])
    sy = int(sy[2:-1])
    bx = int(bx[2:-1])
    by = int(by[2:])
    d = abs(sx-bx) + abs(sy-by)
    sum_d += d
    S.add((sx,sy,d))
    B.add((bx,by))

def valid(x,y,S):
    for (sx,sy,d) in S:
        dxy = abs(x-sx)+abs(y-sy)
        if dxy<=d:
            return False
    return True

p1 = 0
for x in range(-int(1e7),int(1e7)):
    y = int(2e6)
    if not valid(x,y,S) and (x,y) not in B:
        p1 += 1
print(p1)

n_checked = 0
# If there is only one possible position for another beacon, it *must* be distance d+1 from some beacon
# If not, we could find an adjacent position that is possible.
found_p2 = False
for (sx,sy,d) in S:
    # check all points that are d+1 away from (sx,sy)
    for dx in range(d+2):
        dy = (d+1)-dx
        for signx,signy in [(-1,-1),(-1,1),(1,-1),(1,1)]:
            n_checked += 1
            x = sx+(dx*signx)
            y = sy+(dy*signy)
            if not(0<=x<=4000000 and 0<=y<=4000000):
                continue
            assert abs(x-sx)+abs(y-sy)==d+1
            if valid(x,y,S) and (not found_p2):
                print(x*4000000 + y)
                found_p2 = True
#print(n_checked, 4*sum_d) # these are approximately equal

4560025
12480406634249
